In [10]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
#Read the csv files from the given path
df1=spark.read.option("delimiter", ",").option("inferSchema", "true").option("header","true").csv(r"C:\Users\mache\Desktop\SF2010_NO_TNC.csv")
df2=spark.read.option("delimiter", ",").option("inferSchema", "true").option("header","true").csv(r"C:\Users\mache\Desktop\SF2016_NO_TNC.csv")
df3=spark.read.option("delimiter", ",").option("inferSchema", "true").option("header","true").csv(r"C:\Users\mache\Desktop\SF2016_TNC.csv")

#Assign a new column to each file and combine them to one single file
from pyspark.sql.functions import lit
df1=df1.withColumn("file_id",(lit("2010_NOTNC")))
df2=df2.withColumn("file_id",(lit("2016_NOTNC")))
df3=df3.withColumn("file_id",(lit("2016_TNC")))

df = df1.union(df2).union(df3)

In [11]:
#create the SQL view
temp_table_name = "2016_TNC_csv"
df.createOrReplaceTempView(temp_table_name)

In [12]:
#Network Performance Metrics in Base Year, Counterfactual Year 2016 and Actual Year 2016 along with Percent Difference between Base Year and the others
from pyspark import SparkConf,SparkContext as sc
sc = sc.getOrCreate(SparkConf())
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

dframe1 =  sqlContext.sql("""
select file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT),0) as Vehicle_Hours_Traveled, round(sum(VHD),0) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT),0) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD),0) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed 
from `2016_TNC_csv` where file_id = '2010_NOTNC' group by file_id
""");
dframe2 =  sqlContext.sql("""
select file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT),0) as Vehicle_Hours_Traveled, round(sum(VHD),0) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT),0) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD),0) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed  
from `2016_TNC_csv` where file_id = '2016_NOTNC' group by file_id
""");
dframe3 =  sqlContext.sql("""
select file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT),0) as Vehicle_Hours_Traveled, round(sum(VHD),0) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT),0) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD),0) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed  
from `2016_TNC_csv` where file_id = '2016_TNC' group by file_id
""");
dframe = dframe1.union(dframe2).union(dframe3);
import pandas as pd
import numpy as np

pandasDF = dframe.toPandas()
pandasDF = pandasDF.replace(np.nan,'N/A')
pandasDF=pandasDF.astype({'Vehicle_Miles_Traveled':int,'Vehicle_Hours_Traveled':int,'Vehicle_Hours_Delay':int})
print('----------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>92}'.format('Network Performance Metrics'))
print('----------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>55}{:>65}'.format('Based on Modeled Travel Time','Based on Observed Travel Time'))
print('----------------------------------------------------------------------------------------------------------------------------------------------')
pandasDF.set_index("Scenario")

----------------------------------------------------------------------------------------------------------------------------------------------
                                                                 Network Performance Metrics
----------------------------------------------------------------------------------------------------------------------------------------------
                           Based on Modeled Travel Time                                    Based on Observed Travel Time
----------------------------------------------------------------------------------------------------------------------------------------------


,Vehicle_Miles_Traveled,Vehicle_Hours_Traveled,Vehicle_Hours_Delay,Average_Speed,Observed_Vehicle_Hours_Traveled,Observed_Vehicle_Hours_Delay,Observed_Average_Speed
Scenario,,,,,,,
2010_NOTNC,5643247,617271,94256,28.6,742884.0,219870.0,19.1
2016_NOTNC,5941561,667579,115492,28.4,853751.0,301428.0,17.2
2016_TNC,6245800,717041,138701,28.1,895522.0,316933.0,17.2


In [13]:
import pandas as pd
pandasDF = dframe.toPandas()
pandasDF = pandasDF.loc[:, pandasDF.columns != 'Scenario'].pct_change().cumsum()*100
pd.option_context("display.max_rows",100,"display.max_columns",7)
import numpy as np
pandasDF = pandasDF.apply(np.floor).astype(str)+'%'
pandasDF = pandasDF.replace(['nan%'], 0)
print('-----------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>87}'.format('Percent Change from 2010'))
print('-----------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>55}{:>65}'.format('Based on Modeled Travel Time','Based on Observed Travel Time'))
print('-----------------------------------------------------------------------------------------------------------------------------------------------------------')
pandasDF_new=['2010_NOTNC','2016_NOTNC','2016_TNC']
pandasDF.insert(loc=0,column="Scenario",value=pandasDF_new)
pandasDF.set_index("Scenario")

-----------------------------------------------------------------------------------------------------------------------------------------------------------
                                                               Percent Change from 2010
-----------------------------------------------------------------------------------------------------------------------------------------------------------
                           Based on Modeled Travel Time                                    Based on Observed Travel Time
-----------------------------------------------------------------------------------------------------------------------------------------------------------


,Vehicle_Miles_Traveled,Vehicle_Hours_Traveled,Vehicle_Hours_Delay,Average_Speed,Observed_Vehicle_Hours_Traveled,Observed_Vehicle_Hours_Delay,Observed_Average_Speed
Scenario,,,,,,,
2010_NOTNC,0,0,0,0,0,0,0
2016_NOTNC,5.0%,8.0%,22.0%,-1.0%,14.0%,37.0%,-10.0%
2016_TNC,10.0%,15.0%,42.0%,-2.0%,19.0%,42.0%,-10.0%


In [21]:
#Network Performance Metrics in Base Year, Counterfactual Year 2016 and Actual Year 2016 along with Percent Difference between Base Year and the others
dframe1 =  sqlContext.sql("""
select TOD as Time_Of_Day, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed 
from `2016_TNC_csv` where TOD = 'AM' group by file_id, TOD order by file_id
""");
dframe2 =  sqlContext.sql("""
select TOD as Time_Of_Day, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed 
from `2016_TNC_csv` where TOD = 'MD' group by file_id, TOD order by file_id
""");
dframe3 =  sqlContext.sql("""
select TOD as Time_Of_Day, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed 
from `2016_TNC_csv` where  TOD = 'PM' group by file_id, TOD order by file_id
""");
dframe4 = sqlContext.sql("""
select TOD as Time_Of_Day, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed  
from `2016_TNC_csv` where  TOD = 'EV' group by file_id, TOD order by file_id
""");
dframe5 = sqlContext.sql("""
select TOD as Time_Of_Day, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed 
from `2016_TNC_csv` where TOD = 'EA' group by file_id, TOD order by file_id
""");

dframe = dframe1.union(dframe2).union(dframe3).union(dframe4).union(dframe5);

import pyspark.sql.functions as func

dframe = dframe.withColumn("Time_Of_Day", func.when((func.col("Time_Of_Day") == 'AM'), "6-9 AM")
             .when((func.col("Time_Of_Day") == 'MD'), "9 AM-3:30 PM")
             .when((func.col("Time_Of_Day") == 'PM'), "3:30-6:30 PM")
             .when((func.col("Time_Of_Day") == 'EV'), "6:30 PM-3:00 AM")
             .when((func.col("Time_Of_Day") == 'EA'), "3-6 AM")
             .otherwise(0))


dframe = dframe.withColumn("Scenario", func.when((func.col("Scenario") == '2016_TNC'), "2016_WITHTNC")
             .when((func.col("Scenario") == '2010_NOTNC'), "2010_NOTNC")
             .when((func.col("Scenario") == '2016_NOTNC'), "2016_NOTNC")
             .otherwise(0))

import pandas as pd
import numpy as np

pandasDF = dframe.toPandas()
pandasDF = pandasDF.replace(np.nan,'N/A')
pandasDF=pandasDF.astype({'Vehicle_Miles_Traveled':int,'Vehicle_Hours_Traveled':int,'Vehicle_Hours_Delay':int})
print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>100}'.format('Network Performance Metrics'))
print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>55}{:>70}'.format('Based on Modeled Travel Time','Based on Observed Travel Time'))
print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
pandasDF.set_index("Time_Of_Day")

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                         Network Performance Metrics
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                           Based on Modeled Travel Time                                         Based on Observed Travel Time
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Scenario,Vehicle_Miles_Traveled,Vehicle_Hours_Traveled,Vehicle_Hours_Delay,Average_Speed,Observed_Vehicle_Hours_Traveled,Observed_Vehicle_Hours_Delay,Observed_Average_Speed
Time_Of_Day,,,,,,,,
6-9 AM,2010_NOTNC,918794,101643,16147,28.5,120443.0,34948.0,18.4
6-9 AM,2016_NOTNC,972312,110790,20207,28.3,135291.0,44708.0,16.8
6-9 AM,2016_WITHTNC,1006622,116373,22820,28.0,139957.0,46405.0,16.8
9 AM-3:30 PM,2010_NOTNC,2117416,232659,36305,28.0,272576.0,76222.0,18.6
9 AM-3:30 PM,2016_NOTNC,2232077,252796,44916,27.8,315967.0,108087.0,16.5
9 AM-3:30 PM,2016_WITHTNC,2316326,267995,52880,27.3,327968.0,112853.0,16.5
3:30-6:30 PM,2010_NOTNC,1166161,138704,29791,27.7,175080.0,66167.0,17.4
3:30-6:30 PM,2016_NOTNC,1220552,151094,36427,27.4,214626.0,99959.0,15.2
3:30-6:30 PM,2016_WITHTNC,1264435,159755,41184,26.9,221778.0,103206.0,15.2


In [24]:
import pandas as pd
pandasDF = dframe.toPandas()

pandasDF = pandasDF.groupby('Time_Of_Day')['Vehicle_Miles_Traveled', 'Vehicle_Hours_Traveled', 'Vehicle_Hours_Delay', 'Average_Speed', 'Observed_Vehicle_Hours_Traveled', 'Observed_Vehicle_Hours_Delay', 'Observed_Average_Speed'].apply(lambda x: x.div(x.iloc[0]).subtract(1).mul(100).round())

pd.option_context("display.max_rows",100,"display.max_columns",7)
import numpy as np
pandasDF = pandasDF.apply(np.floor).astype(str)+'%'
pandasDF = pandasDF.replace(['nan%'], 'N/A')
print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>87}'.format('Percent Change from 2010'))
print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>55}{:>70}'.format('Based on Modeled Travel Time','Based on Observed Travel Time'))
print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
pandasDF_new=['2010_NOTNC','2016_NOTNC','2016_TNC','2010_NOTNC','2016_NOTNC','2016_TNC','2010_NOTNC','2016_NOTNC','2016_TNC','2010_NOTNC','2016_NOTNC','2016_TNC','2010_NOTNC','2016_NOTNC','2016_TNC']
pandasDF_new1=['6-9 AM','6-9 AM','6-9 AM','9 AM-3:30 PM','9 AM-3:30 PM','9 AM-3:30 PM','3:30-6:30 PM','3:30-6:30 PM','3:30-6:30 PM','6:30 PM-3:00 AM','6:30 PM-3:00 AM','6:30 PM-3:00 AM','3-6 AM','3-6 AM','3-6 AM']


pandasDF.insert(loc=0,column="Time_Of_Day",value=pandasDF_new1)
pandasDF.insert(loc=1,column="Scenario",value=pandasDF_new)
pandasDF.set_index("Time_Of_Day")

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                                                               Percent Change from 2010
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                           Based on Modeled Travel Time                                         Based on Observed Travel Time
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


<ipython-input-24-065397d2a477>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pandasDF = pandasDF.groupby('Time_Of_Day')['Vehicle_Miles_Traveled', 'Vehicle_Hours_Traveled', 'Vehicle_Hours_Delay', 'Average_Speed', 'Observed_Vehicle_Hours_Traveled', 'Observed_Vehicle_Hours_Delay', 'Observed_Average_Speed'].apply(lambda x: x.div(x.iloc[0]).subtract(1).mul(100).round())


,Scenario,Vehicle_Miles_Traveled,Vehicle_Hours_Traveled,Vehicle_Hours_Delay,Average_Speed,Observed_Vehicle_Hours_Traveled,Observed_Vehicle_Hours_Delay,Observed_Average_Speed
Time_Of_Day,,,,,,,,
6-9 AM,2010_NOTNC,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
6-9 AM,2016_NOTNC,6.0%,9.0%,25.0%,-1.0%,12.0%,28.0%,-9.0%
6-9 AM,2016_TNC,10.0%,14.0%,41.0%,-2.0%,16.0%,33.0%,-9.0%
9 AM-3:30 PM,2010_NOTNC,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
9 AM-3:30 PM,2016_NOTNC,5.0%,9.0%,24.0%,-1.0%,16.0%,42.0%,-11.0%
9 AM-3:30 PM,2016_TNC,9.0%,15.0%,46.0%,-3.0%,20.0%,48.0%,-11.0%
3:30-6:30 PM,2010_NOTNC,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
3:30-6:30 PM,2016_NOTNC,5.0%,9.0%,22.0%,-1.0%,23.0%,51.0%,-13.0%
3:30-6:30 PM,2016_TNC,8.0%,15.0%,38.0%,-3.0%,27.0%,56.0%,-13.0%


In [25]:
#Network Performance Metrics in Base Year, Counterfactual Year 2016 and Actual Year 2016 along with Percent Difference between Base Year and the others
dframe1 =  sqlContext.sql("""
select AT as Area_Type, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed 
from `2016_TNC_csv` where AT = 0 group by file_id, AT order by file_id
""");
dframe2 =  sqlContext.sql("""
select AT as Area_Type, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed 
from `2016_TNC_csv` where AT = 1 group by file_id, AT order by file_id
""");
dframe3 =  sqlContext.sql("""
select AT as Area_Type, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed
from `2016_TNC_csv` where AT = 2 group by file_id, AT order by file_id
""");
dframe4 =  sqlContext.sql("""
select AT as Area_Type, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed
from `2016_TNC_csv` where AT = 3 group by file_id, AT order by file_id
""");

dframe = dframe1.union(dframe2).union(dframe3).union(dframe4);


dframe = dframe.withColumn("Area_Type", func.when((func.col("Area_Type") == 0), "Regional Core")
             .when((func.col("Area_Type") == 1), "Central Business District")
             .when((func.col("Area_Type") == 2), "Urban Business")
             .when((func.col("Area_Type") == 3), "Urban")
             .otherwise(0))

import pandas as pd
import numpy as np

pandasDF = dframe.toPandas()
pandasDF = pandasDF.replace(np.nan,'N/A')
pandasDF=pandasDF.astype({'Vehicle_Miles_Traveled':int,'Vehicle_Hours_Traveled':int,'Vehicle_Hours_Delay':int})
print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>100}'.format('Network Performance Metrics'))
print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>55}{:>70}'.format('Based on Modeled Travel Time','Based on Observed Travel Time'))
print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
pandasDF.set_index("Area_Type")

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                         Network Performance Metrics
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                           Based on Modeled Travel Time                                         Based on Observed Travel Time
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Scenario,Vehicle_Miles_Traveled,Vehicle_Hours_Traveled,Vehicle_Hours_Delay,Average_Speed,Observed_Vehicle_Hours_Traveled,Observed_Vehicle_Hours_Delay,Observed_Average_Speed
Area_Type,,,,,,,,
Regional Core,2010_NOTNC,380028,34926,2507,31.5,49747.0,17328.0,14.1
Regional Core,2016_NOTNC,396618,36893,2952,31.3,64139.0,30198.0,11.3
Regional Core,2016_TNC,451716,50216,11252,29.8,73373.0,34410.0,11.3
Central Business District,2010_NOTNC,1851831,245541,39701,28.9,290690.0,84850.0,17.4
Central Business District,2016_NOTNC,1953974,270108,52476,28.7,339622.0,121981.0,15.0
Central Business District,2016_TNC,2064224,284108,57553,28.2,353993.0,127427.0,15.0
Urban Business,2010_NOTNC,1961013,183566,34989,27.3,207460.0,58884.0,22.2
Urban Business,2016_NOTNC,2066714,199173,41521,27.0,247378.0,89726.0,20.5
Urban Business,2016_TNC,2160125,214681,49304,27.3,259207.0,93830.0,20.5


In [26]:
import pandas as pd
pandasDF = dframe.toPandas()

pandasDF = pandasDF.groupby('Area_Type')['Vehicle_Miles_Traveled', 'Vehicle_Hours_Traveled', 'Vehicle_Hours_Delay', 'Average_Speed', 'Observed_Vehicle_Hours_Traveled', 'Observed_Vehicle_Hours_Delay', 'Observed_Average_Speed'].apply(lambda x: x.div(x.iloc[0]).subtract(1).mul(100).round())

pd.option_context("display.max_rows",100,"display.max_columns",7)
import numpy as np
pandasDF = pandasDF.apply(np.floor).astype(str)+'%'
pandasDF = pandasDF.replace(['nan%'], 'N/A')
print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>95}'.format('Percent Change from 2010'))
print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>55}{:>70}'.format('Based on Modeled Travel Time','Based on Observed Travel Time'))
print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
pandasDF_new =['2010_NOTNC','2016_NOTNC','2016_TNC','2010_NOTNC','2016_NOTNC','2016_TNC','2010_NOTNC','2016_NOTNC','2016_TNC','2010_NOTNC','2016_NOTNC','2016_TNC']
pandasDF_new1=['Regional Core','Regional Core','Regional Core','Central Business District','Central Business District','Central Business District','Urban Business','Urban Business','Urban Business','Urban','Urban','Urban']
pandasDF.insert(loc=0,column="Area_Type",value=pandasDF_new1)
pandasDF.insert(loc=1,column="Scenario",value=pandasDF_new)
pandasDF.set_index("Area_Type")

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                       Percent Change from 2010
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                           Based on Modeled Travel Time                                         Based on Observed Travel Time
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


<ipython-input-26-c8f3f64724c7>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pandasDF = pandasDF.groupby('Area_Type')['Vehicle_Miles_Traveled', 'Vehicle_Hours_Traveled', 'Vehicle_Hours_Delay', 'Average_Speed', 'Observed_Vehicle_Hours_Traveled', 'Observed_Vehicle_Hours_Delay', 'Observed_Average_Speed'].apply(lambda x: x.div(x.iloc[0]).subtract(1).mul(100).round())


,Scenario,Vehicle_Miles_Traveled,Vehicle_Hours_Traveled,Vehicle_Hours_Delay,Average_Speed,Observed_Vehicle_Hours_Traveled,Observed_Vehicle_Hours_Delay,Observed_Average_Speed
Area_Type,,,,,,,,
Regional Core,2010_NOTNC,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Regional Core,2016_NOTNC,4.0%,6.0%,18.0%,-1.0%,29.0%,74.0%,-20.0%
Regional Core,2016_TNC,19.0%,44.0%,349.0%,-5.0%,47.0%,99.0%,-20.0%
Central Business District,2010_NOTNC,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Central Business District,2016_NOTNC,6.0%,10.0%,32.0%,-1.0%,17.0%,44.0%,-14.0%
Central Business District,2016_TNC,11.0%,16.0%,45.0%,-2.0%,22.0%,50.0%,-14.0%
Urban Business,2010_NOTNC,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Urban Business,2016_NOTNC,5.0%,9.0%,19.0%,-1.0%,19.0%,52.0%,-8.0%
Urban Business,2016_TNC,10.0%,17.0%,41.0%,0.0%,25.0%,59.0%,-8.0%


In [27]:
#Network Performance Metrics in Base Year, Counterfactual Year 2016 and Actual Year 2016 along with Percent Difference between Base Year and the others
dframe1 =  sqlContext.sql("""
select FT2 as Facility_Type, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed 
from `2016_TNC_csv` where FT2 = 1 group by file_id, FT2 order by file_id
""");
dframe2 =   sqlContext.sql("""
select FT2 as Facility_Type, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed 
from `2016_TNC_csv` where FT2 = 2 group by file_id, FT2 order by file_id
""");
dframe3 =  sqlContext.sql("""
select FT2 as Facility_Type, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed  
from `2016_TNC_csv` where FT2 = 3 group by file_id, FT2 order by file_id
""");
dframe4 = sqlContext.sql("""
select FT2 as Facility_Type, file_id as Scenario, round(sum(VMT)) as Vehicle_Miles_Traveled, round(sum(VHT)) as Vehicle_Hours_Traveled, round(sum(VHD)) as Vehicle_Hours_Delay, round(avg(SPEED),1) as Average_Speed, 
round(sum(OBS_VHT)) as Observed_Vehicle_Hours_Traveled, round(sum(OBS_VHD)) as Observed_Vehicle_Hours_Delay, round(avg(OBS_SPEED),1) as Observed_Average_Speed 
from `2016_TNC_csv` where FT2 = 4 group by file_id, FT2 order by file_id
""");

dframe = dframe1.union(dframe2).union(dframe3).union(dframe4);


dframe = dframe.withColumn("Facility_Type", func.when((func.col("Facility_Type") == 1), "Freeways & Ramps")
             .when((func.col("Facility_Type") == 2), "Major Arterial")
             .when((func.col("Facility_Type") == 3), "Minor Arterial")
             .when((func.col("Facility_Type") == 4), "Collectors & Locals")
             .otherwise(0))

import pandas as pd
import numpy as np

pandasDF = dframe.toPandas()
pandasDF = pandasDF.replace(np.nan,'N/A')
pandasDF=pandasDF.astype({'Vehicle_Miles_Traveled':int,'Vehicle_Hours_Traveled':int,'Vehicle_Hours_Delay':int})
print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>100}'.format('Network Performance Metrics'))
print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>55}{:>70}'.format('Based on Modeled Travel Time','Based on Observed Travel Time'))
print('----------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
pandasDF.set_index("Facility_Type")


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                         Network Performance Metrics
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                           Based on Modeled Travel Time                                         Based on Observed Travel Time
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Scenario,Vehicle_Miles_Traveled,Vehicle_Hours_Traveled,Vehicle_Hours_Delay,Average_Speed,Observed_Vehicle_Hours_Traveled,Observed_Vehicle_Hours_Delay,Observed_Average_Speed
Facility_Type,,,,,,,,
Freeways & Ramps,2010_NOTNC,2926552,349599,79006,28.0,372448.0,101854.0,47.6
Freeways & Ramps,2016_NOTNC,3081275,382145,97191,27.8,421815.0,136625.0,47.2
Freeways & Ramps,2016_TNC,3158598,394843,104837,27.6,428783.0,138527.0,47.2
Major Arterial,2010_NOTNC,1938331,176867,10551,34.0,245907.0,79592.0,19.7
Major Arterial,2016_NOTNC,2039947,189063,12860,33.7,288139.0,111937.0,17.6
Major Arterial,2016_TNC,2190202,214481,25147,33.3,309876.0,120542.0,17.6
Minor Arterial,2010_NOTNC,527670,62033,3996,28.6,85324.0,27287.0,15.0
Minor Arterial,2016_NOTNC,551241,65578,4640,28.5,97321.0,36382.0,13.0
Minor Arterial,2016_TNC,600089,73267,7405,27.8,105454.0,39592.0,13.0


In [28]:
import pandas as pd
pandasDF = dframe.toPandas()
pandasDF
pandasDF = pandasDF.groupby('Facility_Type')['Vehicle_Miles_Traveled', 'Vehicle_Hours_Traveled', 'Vehicle_Hours_Delay', 'Average_Speed', 'Observed_Vehicle_Hours_Traveled', 'Observed_Vehicle_Hours_Delay', 'Observed_Average_Speed'].apply(lambda x: x.div(x.iloc[0]).subtract(1).mul(100))

pd.option_context("display.max_rows",100,"display.max_columns",7)
import numpy as np
pandasDF = pandasDF.apply(np.floor).astype(str)+'%'
pandasDF = pandasDF.replace(['nan%'], 'N/A')
print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>95}'.format('Percent Change from 2010'))
print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('{:>55}{:>70}'.format('Based on Modeled Travel Time','Based on Observed Travel Time'))
print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
pandasDF_new =['2010_NOTNC','2016_NOTNC','2016_TNC','2010_NOTNC','2016_NOTNC','2016_TNC','2010_NOTNC','2016_NOTNC','2016_TNC','2010_NOTNC','2016_NOTNC','2016_TNC']
pandasDF_new1=['Freeways & Ramps','Freeways & Ramps','Freeways & Ramps','Major Arterial','Major Arterial','Major Arterial','Minor Arterial','Minor Arterial','Minor Arterial','Collectors & Locals','Collectors & Locals','Collectors & Locals']
pandasDF.insert(loc=0,column="Facility Type",value=pandasDF_new1)
pandasDF.insert(loc=1,column="Scenario",value=pandasDF_new)
pandasDF.set_index("Facility Type")

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                                                                       Percent Change from 2010
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                           Based on Modeled Travel Time                                         Based on Observed Travel Time
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


<ipython-input-28-58933dba62b8>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pandasDF = pandasDF.groupby('Facility_Type')['Vehicle_Miles_Traveled', 'Vehicle_Hours_Traveled', 'Vehicle_Hours_Delay', 'Average_Speed', 'Observed_Vehicle_Hours_Traveled', 'Observed_Vehicle_Hours_Delay', 'Observed_Average_Speed'].apply(lambda x: x.div(x.iloc[0]).subtract(1).mul(100))


,Scenario,Vehicle_Miles_Traveled,Vehicle_Hours_Traveled,Vehicle_Hours_Delay,Average_Speed,Observed_Vehicle_Hours_Traveled,Observed_Vehicle_Hours_Delay,Observed_Average_Speed
Facility Type,,,,,,,,
Freeways & Ramps,2010_NOTNC,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Freeways & Ramps,2016_NOTNC,5.0%,9.0%,23.0%,-1.0%,13.0%,34.0%,-1.0%
Freeways & Ramps,2016_TNC,7.0%,12.0%,32.0%,-2.0%,15.0%,36.0%,-1.0%
Major Arterial,2010_NOTNC,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Major Arterial,2016_NOTNC,5.0%,6.0%,21.0%,-1.0%,17.0%,40.0%,-11.0%
Major Arterial,2016_TNC,12.0%,21.0%,138.0%,-3.0%,26.0%,51.0%,-11.0%
Minor Arterial,2010_NOTNC,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
Minor Arterial,2016_NOTNC,4.0%,5.0%,16.0%,-1.0%,14.0%,33.0%,-14.0%
Minor Arterial,2016_TNC,13.0%,18.0%,85.0%,-3.0%,23.0%,45.0%,-14.0%
